In [1]:
from typing import Tuple, Union

import btrdb

from btrdb.stream import Stream

from btrdb.utils.general import pointwidth
from btrdb.utils.timez import ns_to_datetime, to_nanoseconds

conn = btrdb.connect()

In [2]:
streams = conn.streams_in_collection("sunshine", tags={"unit": "volts"})
stream = streams[0]
stream.uuid

UUID('d60fc469-a6da-4c98-8763-fd833293d955')

In [3]:
# initializing the start, end, and pointwidth
start = stream.earliest()[0].time
end = stream.latest()[0].time
initial_pw: int = pointwidth.from_nanoseconds(end - start)
version = 0
start, end, initial_pw

(1456790400008333000, 1464738830333333000, <pointwidth 52>)

### Search for a measurement equals to specific value
To reduce the search, the equality is limited by the specified global aggregation `min` or `max` especially works well. 
This would be mostly used for identically 0.0 values. 

In [4]:
def search_timestamps_at_agg_value(
    stream: Stream,
    value: Union[int, float],
    agg: str,
    start: int,
    end: int,
    tol=1e-3,
    initial_pw: int = 49,
    final_pw: int = 36,
    return_rawpoint_timestamps: bool = False,
    version: int = 0,
) -> tuple:
    """
    Find points in stream between `start` and `end` that are equal to the specified `threshold`
    using StatPoints recursively through BTrDB tree.
    Parameters
    ----------
    stream: Stream
        Stream to search
    value: tuple of (agg,threshold)
        Find values with given aggregate and threshold value.
    start: int
        The start time in nanoseconds for the range to search from.
    end: int
        The end time in nanoseconds for the range to search from.
    initial_pw: int or pointwidth, default: 49
        Initial query pointwidth of tree traversal, Default is 49 (approximately 7 days).
    final_pw: int, default: 36
        Final pointwidth depth to use tree traversal with StatPoints and to search with RawPoints. Default is 36
        (approximately 1.15 minutes).
    return_rawpoint_timestamps: bool, default: False
        Return RawPoint timestamps if `True`, else returns time-range tuples of start and end timestamps of the
        StatPoints windows that is equal to threshold.
    version: int, default: 0
        Version of the stream to search from.
    Yields
    ------
    tuple
        Timestamp (nanoseconds) of start (and end if equal to threshold for more than specified max depth, default
        ~1.15 minutes) timestamps of event.
    """
    all_measure_aggs = ["min", "mean", "max"]
    assert isinstance(
        initial_pw, (int, pointwidth)
    ), "Please provide `initial_pw` as an integer or pointwidth object"
    if isinstance(initial_pw, int):
        initial_pw = pointwidth(
            initial_pw
        )  # convert initial_pw integer to pointwidth object
    stack = [(start, end, initial_pw)]
    while stack:
        wstart, wend, pw = stack.pop()
        windows = stream.arrow_aligned_windows(
            wstart, wend, int(pw), version
        ).to_pylist()
        for window in windows:
            wstart = window["time"].value
            wend = wstart + pw.nanoseconds
            if not return_rawpoint_timestamps and all(
                [window[_agg] == value for _agg in all_measure_aggs]
            ):
                # only returns the window if the window's mean is close to value and stddev are smaller than 2x tolerance
                yield (wstart, wend)
            elif (value - tol) <= window[agg] <= (value + tol):
                # If we are at a window length of a max_depth, use values
                if pw <= final_pw and not return_rawpoint_timestamps:
                    points = []
                    yield (wstart, wend)
                elif return_rawpoint_timestamps and pw <= final_pw:
                    points = stream.arrow_values(wstart, wend, version).to_pylist()
                else:
                    stack.append((wstart, wend, pw - 2))
                    continue

                for point in points:
                    if isinstance(point, dict):
                        if value - tol <= point["value"] <= value + tol:
                            yield (point["time"].value,)
                    else:
                        yield point

In [5]:
detect_generator = search_timestamps_at_agg_value(
    stream, 240.4445343017578, "min", start, end, initial_pw=initial_pw
)
timestamps = [
    (ns_to_datetime(start).isoformat(), ns_to_datetime(end).isoformat())
    for start, end in detect_generator
]
timestamps

[('2016-03-15T22:16:07.938228+00:00', '2016-03-15T22:17:16.657705+00:00')]

In [6]:
detect_generator = search_timestamps_at_agg_value(
    stream, 293.46502685546875, "max", start, end, initial_pw=initial_pw
)
timestamps = [
    (ns_to_datetime(start).isoformat(), ns_to_datetime(end).isoformat())
    for start, end in detect_generator
]
timestamps

[('2016-04-19T13:21:16.516360+00:00', '2016-04-19T13:22:25.235837+00:00')]

In [11]:
# get the rawpoints timestamp, not a range:
detect_generator = search_timestamps_at_agg_value(
    stream, 240.444534, "min", start, end, tol=1e-1, return_rawpoint_timestamps=True
)
timestamps = [time[0] for time in detect_generator]
print(timestamps)

# check if the point is the `min` value of `240.4445343017578`
event_time = timestamps[0]
stream.arrow_values(event_time, event_time + 1)

[1458080218574999000]


pyarrow.Table
time: timestamp[ns, tz=UTC] not null
value: double not null
----
time: [[2016-03-15 22:16:58.574999000]]
value: [[240.4445343017578]]

#
If the value is not a global min or max still need implementation and returns an error. we have detectors

In [12]:
detect_generator = search_timestamps_at_agg_value(
    stream, 250.444534, "min", start, end, tol=1e-1, return_rawpoint_timestamps=True
)
timestamps = [time[0] for time in detect_generator]
print(timestamps)

[]
